In [2]:
!unzip -uq "/content/drive/MyDrive/gpu hackathon/test.zip"
!unzip -uq "/content/drive/MyDrive/gpu hackathon/training.zip"

In [3]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator,load_img
#from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [4]:
Image_Width=64
Image_Height=64
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='binary_crossentropy',
  optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 29, 29, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0

In [6]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [26]:
train = ImageDataGenerator(rescale=1/255,validation_split=0.2)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("/content/drive/MyDrive/gpu hackathon/training.zip (Unzipped Files)/training/",
                                          target_size=(64,64),
                                          batch_size = 32,
                                          subset='training')
validation_dataset = train.flow_from_directory("/content/drive/MyDrive/gpu hackathon/training.zip (Unzipped Files)/training/",
                                          target_size=(64,64),
                                          batch_size = 32,
                                          subset='validation')
                                         
test_dataset = test.flow_from_directory( "/content/test",
                                          classes=[''],
                                          shuffle=False
                                          )

Found 4700 images belonging to 2 classes.
Found 1175 images belonging to 2 classes.
Found 98 images belonging to 1 classes.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
epochs=100
history = model.fit_generator(
    train_dataset, 
    epochs=epochs,
    validation_data=validation_dataset,
    validation_steps=len(validation_dataset),
    steps_per_epoch=len(train_dataset),
    callbacks=callbacks
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
147/147 [==============================] - 1630s 11s/step - loss: 0.5315 - accuracy: 0.7940 - val_loss: 1.6270 - val_accuracy: 0.7574
Epoch 2/15
147/147 [==============================] - 53s 361ms/step - loss: 0.2526 - accuracy: 0.9111 - val_loss: 0.9689 - val_accuracy: 0.7617
Epoch 3/15
147/147 [==============================] - 54s 366ms/step - loss: 0.1982 - accuracy: 0.9355 - val_loss: 0.3582 - val_accuracy: 0.8732
Epoch 4/15
147/147 [==============================] - 54s 365ms/step - loss: 0.1620 - accuracy: 0.9430 - val_loss: 0.2360 - val_accuracy: 0.9055
Epoch 5/15
147/147 [==============================] - 53s 361ms/step - loss: 0.1274 - accuracy: 0.9572 - val_loss: 0.1844 - val_accuracy: 0.9200
Epoch 6/15
147/147 [==============================] - 54s 366ms/step - loss: 0.0959 - accuracy: 0.9668 - val_loss: 0.2034 - val_accuracy: 0.9157
Epoch 7/15
147/147 [==============================] - 53s 358ms/step - loss: 0.0897 - accuracy: 0.9672 - val_loss: 0.2441 - val_ac

In [9]:
model.save("97.95_model.h5")

In [47]:
import os
images = []
labels=(np.array(os.listdir('/content/test/'))).T
for i in range(len(labels)):
  labels[i]=labels[i].split('.')[0]
labels=labels.astype(int)

for img in os.listdir('/content/test/'):
    img = load_img('/content/test/'+img)
    img = np.array(img)
    img=img/255
    img=np.expand_dims(img,axis=1)
    images.append(img.reshape(1,64,64,3))
images = np.vstack(images)
classes = ((model.predict(images).argmax(axis=1))).T

final=(np.vstack((labels,classes))).T
final = final[final[:, 0].argsort()]
print(final)

[[ 1  1]
 [ 2  0]
 [ 3  1]
 [ 4  0]
 [ 5  1]
 [ 6  0]
 [ 7  1]
 [ 8  0]
 [ 9  1]
 [10  0]
 [11  1]
 [12  0]
 [13  1]
 [14  0]
 [15  1]
 [16  0]
 [17  1]
 [18  0]
 [19  1]
 [20  1]
 [21  1]
 [22  1]
 [23  1]
 [24  1]
 [25  1]
 [26  1]
 [27  0]
 [28  1]
 [29  1]
 [30  1]
 [31  0]
 [32  0]
 [33  0]
 [34  0]
 [35  0]
 [36  0]
 [37  0]
 [38  0]
 [39  0]
 [40  1]
 [41  0]
 [42  1]
 [43  0]
 [44  1]
 [45  0]
 [46  1]
 [47  0]
 [48  1]
 [49  0]
 [50  1]
 [51  1]
 [52  1]
 [53  1]
 [54  1]
 [55  1]
 [56  0]
 [57  0]
 [58  0]
 [59  0]
 [60  0]
 [61  1]
 [62  1]
 [63  1]
 [64  0]
 [65  0]
 [66  0]
 [67  1]
 [68  1]
 [69  1]
 [70  1]
 [71  1]
 [72  1]
 [73  1]
 [74  0]
 [75  0]
 [76  0]
 [77  0]
 [78  0]
 [79  0]
 [80  0]
 [81  1]
 [82  1]
 [83  1]
 [84  1]
 [85  0]
 [86  0]
 [87  0]
 [88  0]
 [89  0]
 [90  1]
 [91  1]
 [92  1]
 [93  1]
 [94  1]
 [95  0]
 [96  0]
 [97  0]
 [98  0]]


In [48]:
import random
import json


def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

def read_json(filename):
    with open(filename, 'r') as outfile:
        data =  json.load(outfile)
    return data

def generate_sample_file(filename):
    res = {}
    for i in final:
        test_set = str(i[0]) + '.jpg'
        res[test_set] = int(i[1])

    write_json(filename, res)

if __name__ == '__main__':
    generate_sample_file('./sample_result1.json')
